In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore')

In [2]:
train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

In [21]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0
total_accuracy2 = 0

# Define the parameter grid for the search
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 6, 10],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Create an empty list to store the best parameters for each question
best_params_list = []

for q_no in range(1, 19):
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Create the XGBoost model
    model = xgb.XGBClassifier(eval_metric='error', use_label_encoder=False)

    # Create the RandomizedSearchCV object
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=20, cv=3, scoring='accuracy', n_jobs=-1, verbose=0, random_state=42)

    # Fit the RandomizedSearchCV object to the data
    random_search.fit(train_X, train_y)


    best_params = random_search.best_params_
    best_params_list.append(best_params)
    print(f"Best hyperparameters for question {q_no}:", best_params)

    # Get the best estimator and fit it to the data
    best_model = random_search.best_estimator_
    best_model.fit(train_X, train_y)

       # Calculate the accuracy and F1 score using cross_val_score
    question_accuracy = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5))
    question_accuracy2 = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5, scoring='f1'))
    total_accuracy += question_accuracy
    total_accuracy2 += question_accuracy2
    print("Question", q_no, "accuracy:", question_accuracy, "f1:", question_accuracy2)


overall_accuracy = total_accuracy / 18
overall_accuracy2 = total_accuracy2 / 18
print("Overall accuracy:", overall_accuracy, "Overall f1:", overall_accuracy2)


Best hyperparameters for question 1: {'subsample': 1, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 1}
Question 1 accuracy: 0.7273500347853105 f1: 0.8420765776953882
Best hyperparameters for question 2: {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.8}
Question 2 accuracy: 0.980267610485576 f1: 0.9900354266090516
Best hyperparameters for question 3: {'subsample': 1, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 1}
Question 3 accuracy: 0.9359220809045532 f1: 0.9669005007379587
Best hyperparameters for question 4: {'subsample': 0.8, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 1}
Question 4 accuracy: 0.7901545188257199 f1: 0.8827222850315637
Best hyperparameters for question 5: {'subsample

In [24]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0
total_accuracy2 = 0

# Create an empty list to store the best parameters for each question
best_params_list_randomforest = []

for q_no in range(1, 19):
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    param_dist = {
        'n_estimators': randint(100, 500),
        'max_depth': randint(1, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'criterion': ['gini', 'entropy'],
    }

    base_model = RandomForestClassifier()

    random_search = RandomizedSearchCV(base_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)

    random_search.fit(train_X, train_y)

    best_params = random_search.best_params_
    best_params_list_randomforest.append(best_params)
    print(f"Best hyperparameters for question {q_no}:", best_params)

    # Get the best estimator and fit it to the data
    best_model = random_search.best_estimator_
    best_model.fit(train_X, train_y)

       # Calculate the accuracy and F1 score using cross_val_score
    question_accuracy = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5))
    question_accuracy2 = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5, scoring='f1'))
    total_accuracy += question_accuracy
    total_accuracy2 += question_accuracy2
    print("Question", q_no, "accuracy:", question_accuracy, "f1:", question_accuracy2)


overall_accuracy = total_accuracy / 18
overall_accuracy2 = total_accuracy2 / 18
print("Overall accuracy:", overall_accuracy, "Overall f1:", overall_accuracy2)



Best hyperparameters for question 1: {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 237}
Question 1 accuracy: 0.7235308553584012 f1: 0.8395913603637742
Best hyperparameters for question 2: {'criterion': 'entropy', 'max_depth': 37, 'min_samples_leaf': 11, 'min_samples_split': 12, 'n_estimators': 312}
Question 2 accuracy: 0.9779335048958353 f1: 0.9888436167402521
Best hyperparameters for question 3: {'criterion': 'gini', 'max_depth': 29, 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 135}
Question 3 accuracy: 0.9310417806476596 f1: 0.9642896281516811
Best hyperparameters for question 4: {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 7, 'n_estimators': 351}
Question 4 accuracy: 0.8016127325493694 f1: 0.8900139804255083
Best hyperparameters for question 5: {'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 19, 'min_samples_split': 7, 'n_estimators': 331}
Question 5 accura

In [25]:
import pandas as pd
from catboost import CatBoostClassifier
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

# Assuming you have already read and preprocessed the data
train = pd.read_csv('CatBoostData.csv')
labels = pd.read_csv('train_labels.csv')

# Specify the categorical columns to be used by the CatBoost model
cat_features = ['event_name_mode', 'fqid_mode', 'room_fqid_mode', 'text_mode']

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0
total_accuracy2 = 0

# Create an empty list to store the best parameters for each question
best_params_list_catboost = []

# Hyperparameter tuning configuration
param_dist = {
    'iterations': [100, 300, 500, 700],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'depth': [3, 4, 6, 8],
    'verbose': [0]
}

# Loop through questions, create train & validation sets for each question
for q_no in range(1, 19):

    # Filter dataset based on current question
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    # Create train & validation sets
    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Hyperparameter tuning for the CatBoost model
    catboost_base_model = CatBoostClassifier(cat_features=cat_features)
    catboost_random_search = RandomizedSearchCV(catboost_base_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
    catboost_random_search.fit(train_X, train_y)
   
    best_params = catboost_random_search.best_params_
    best_params_list_catboost.append(best_params)
    print(f"Best hyperparameters for question {q_no}:", best_params)

    # Get the best estimator and fit it to the data
    best_model = catboost_random_search.best_estimator_
    best_model.fit(train_X, train_y)

       # Calculate the accuracy and F1 score using cross_val_score
    question_accuracy = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5))
    question_accuracy2 = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5, scoring='f1'))
    total_accuracy += question_accuracy
    total_accuracy2 += question_accuracy2
    print("Question", q_no, "accuracy:", question_accuracy, "f1:", question_accuracy2)


overall_accuracy = total_accuracy / 18
overall_accuracy2 = total_accuracy2 / 18
print("Overall accuracy:", overall_accuracy, "Overall f1:", overall_accuracy2)


Best hyperparameters for question 1: {'verbose': 0, 'learning_rate': 0.01, 'iterations': 700, 'depth': 6}
Question 1 accuracy: 0.7207723464661951 f1: 0.8377311467489829
Best hyperparameters for question 2: {'verbose': 0, 'learning_rate': 0.1, 'iterations': 700, 'depth': 8}
Question 2 accuracy: 0.9779335048958353 f1: 0.9888436167402521
Best hyperparameters for question 3: {'verbose': 0, 'learning_rate': 0.01, 'iterations': 500, 'depth': 8}
Question 3 accuracy: 0.9363462590593782 f1: 0.9671268807286563
Best hyperparameters for question 4: {'verbose': 0, 'learning_rate': 0.01, 'iterations': 100, 'depth': 8}
Question 4 accuracy: 0.8003395226419725 f1: 0.8890983781816713
Best hyperparameters for question 5: {'verbose': 0, 'learning_rate': 0.01, 'iterations': 100, 'depth': 3}
Question 5 accuracy: 0.5499681416088601 f1: 0.7096508152555747
Best hyperparameters for question 6: {'verbose': 0, 'learning_rate': 0.05, 'iterations': 300, 'depth': 4}
Question 6 accuracy: 0.7778483990877018 f1: 0.8749

In [15]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
warnings.simplefilter(action='ignore')

# Assuming you have already read and preprocessed the data
train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0
total_accuracy2 = 0

# Define the parameter grid for the search
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 6, 10],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Create an empty list to store the best parameters for each question
best_params_list_lightGBM = []

# Loop through questions, create train & validation sets for each question
for q_no in range(1, 19):

    # Filter dataset based on current question
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    # Create train & validation sets
    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Create the LightGBM model
    model = LGBMClassifier()

    # Create the RandomizedSearchCV object
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=20, cv=3, scoring='accuracy', n_jobs=-1, verbose=0)

    # Fit the RandomizedSearchCV object to the data
    random_search.fit(train_X, train_y)

    best_params = random_search.best_params_
    best_params_list_lightGBM.append(best_params)
    print(f"Best hyperparameters for question {q_no}:", best_params)

    # Get the best estimator and fit it to the data
    best_model = random_search.best_estimator_
    best_model.fit(train_X, train_y)

       # Calculate the accuracy and F1 score using cross_val_score
    question_accuracy = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5))
    question_accuracy2 = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5, scoring='f1'))
    total_accuracy += question_accuracy
    total_accuracy2 += question_accuracy2
    print("Question", q_no, "accuracy:", question_accuracy, "f1:", question_accuracy2)


overall_accuracy = total_accuracy / 18
overall_accuracy2 = total_accuracy2 / 18
print("Overall accuracy:", overall_accuracy, "Overall f1:", overall_accuracy2)

Question 1 accuracy: 0.726500327589817 f1: 0.8411191225949011
Question 2 accuracy: 0.9781458191208884 f1: 0.988952122334856
Question 3 accuracy: 0.9333758862374001 f1: 0.9655399647513658
Question 4 accuracy: 0.8047947441534786 f1: 0.8916885854461937
Question 5 accuracy: 0.5032916585050647 f1: 0.606575882320669
Question 6 accuracy: 0.7693610084813117 f1: 0.8694287893751348
Question 7 accuracy: 0.7279867523128292 f1: 0.8423101254175558
Question 8 accuracy: 0.5979189603582549 f1: 0.7373968299111142
Question 9 accuracy: 0.7405042856853381 f1: 0.8504285894327992
Question 10 accuracy: 0.5151675211019626 f1: 0.5085124963333041
Question 11 accuracy: 0.6373864411587898 f1: 0.7724626938769334
Question 12 accuracy: 0.8688728883965661 f1: 0.9298193276627499
Question 13 accuracy: 0.7224690590854953 f1: 0.02374637010006861
Question 14 accuracy: 0.7029487586484839 f1: 0.8246906710637946
Question 15 accuracy: 0.5094377387972162 f1: 0.4357590685183494
Question 16 accuracy: 0.739657505409172 f1: 0.85012

In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
warnings.simplefilter(action='ignore')

train_other = pd.read_csv('StudentPerformancePred.csv')
train_catboost = pd.read_csv('CatBoostData.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]))
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]))

train_other['session_id'] = train_other['session_id'].astype(str)
train_catboost['session_id'] = train_catboost['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

cat_features = ['event_name_mode', 'fqid_mode', 'room_fqid_mode', 'text_mode']

total_accuracy = 0

for q_no in range(1, 19):
    if q_no <= 3:
        select_other = train_other[train_other['level_group'] == '0-4']
        select_catboost = train_catboost[train_catboost['level_group'] == '0-4']
    elif q_no <= 13:
        select_other = train_other[train_other['level_group'] == '5-12']
        select_catboost = train_catboost[train_catboost['level_group'] == '5-12']
    else:
        select_other = train_other[train_other['level_group'] == '13-22']
        select_catboost = train_catboost[train_catboost['level_group'] == '13-22']

    train_X_other, validate_X_other = train_test_split(select_other, test_size=0.2)
    train_X_catboost, validate_X_catboost = train_test_split(select_catboost, test_size=0.2)

    train_y = labels.loc[(labels['session'].isin(train_X_other['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X_other['session_id'])) & (labels['q'] == q_no)]

    train_X_other = train_X_other.drop(['session_id', 'level_group'], axis=1)
    train_X_catboost = train_X_catboost.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X_other = validate_X_other.drop(['session_id', 'level_group'], axis=1)
    validate_X_catboost = validate_X_catboost.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Train the four models
    rf_model = RandomForestClassifier(n_estimators=100)
    xgb_model = XGBClassifier()
    cat_model = CatBoostClassifier(cat_features=cat_features, iterations=500, learning_rate=0.1, depth=6, verbose=0)
    lgbm_model = LGBMClassifier()

    # Create the voting classifier
    voting_classifier = VotingClassifier(
        estimators=[('rf', rf_model), ('xgb', xgb_model), ('cat', cat_model), ('lgbm', lgbm_model)],
        voting='soft'
    )

    rf_model.fit(train_X_other, train_y)
    xgb_model.fit(train_X_other, train_y)
    cat_model.fit(train_X_catboost, train_y)
    lgbm_model.fit(train_X_other, train_y)

    # Obtain the predictions
    rf_pred = rf_model.predict_proba(validate_X_other)[:, 1]
    xgb_pred = xgb_model.predict_proba(validate_X_other)[:, 1]
    cat_pred = cat_model.predict_proba(validate_X_catboost)[:, 1]
    lgbm_pred = lgbm_model.predict_proba(validate_X_other)[:, 1]

    # Average the predictions
    ensemble_pred = (rf_pred + xgb_pred + cat_pred + lgbm_pred) / 4
    ensemble_pred = np.round(ensemble_pred).astype(int)

    # Calculate the accuracy of the ensemble model
    question_accuracy = np.mean(ensemble_pred == validate_y)
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.7165287502652239
Question 2 accuracy: 0.9777211966900063
Question 3 accuracy: 0.9295565457245916
Question 4 accuracy: 0.80500742626777
Question 5 accuracy: 0.539147040101846
Question 6 accuracy: 0.7644812221514958
Question 7 accuracy: 0.7358370464672184
Question 8 accuracy: 0.6040738383195416
Question 9 accuracy: 0.7265011669849353
Question 10 accuracy: 0.5128368342881392
Question 11 accuracy: 0.6346276257161044
Question 12 accuracy: 0.8663271801400382
Question 13 accuracy: 0.7224697644812221
Question 14 accuracy: 0.7008274984086569
Question 15 accuracy: 0.4988330150647146
Question 16 accuracy: 0.7388075535752174
Question 17 accuracy: 0.6893698281349459
Question 18 accuracy: 0.954169318905156
Overall accuracy: 0.7287290473159348
